# Grouping angensts with a supervisor
Use an LLM to orchestrate and delegate to individual agents

In [3]:
%%capture --no-stderr
%pip install -U langchain_ollama

In [2]:
import getpass
import os


def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}")


_set_if_undefined("TAVILY_API_KEY")


## Create tools
An agent to do web research with a search engine, and one agent to create plots

In [4]:
from typing import Annotated

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_experimental.tools import PythonAstREPLTool

tavily_tool = TavilySearchResults(max_results=5)

python_repl_tool = PythonAstREPLTool()
